# Звіт до лабораторної роботи №4 (Сенчак Павло ОІ-21сп)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [3]:
import warnings
warnings.simplefilter('ignore')

In [4]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

### Read the dataset for churn 

In [5]:
df = pd.read_csv("new_variant_1.csv")

# Model tuning


#### Parameters vs Hyperparameters
Параметр можна вважати внутрішнім для моделі та його можна отримати після того, як модель навчилася на даних. Прикладами параметрів є коефіцієнти регресії в лінійній регресії, опорні вектори в машинах опорних векторів та ваги в нейронних мережах.

Гіперпараметр можна вважати зовнішнім для моделі та його може встановлювати фахівець на власний розсуд. Прикладами гіперпараметрів є k в алгоритмі k-ближчих сусідів, кількість дерев та максимальна кількість ознак в алгоритмі випадкового лісу, швидкість навчання та момент в нейронних мережах, параметри C та гамма в машинах опорних векторів.

#### Define target and features columns

In [6]:
X = df.drop(columns='Status')
y = df['Status']

In [7]:
# Let's say we want to split the data in 80:10:10 for train:valid:test dataset
# In the first step we will split the data in training and remaining dataset
X_train, X_test, y_train, y_test = train_test_split(X,y, train_size=0.8)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


(115318, 31)
(115318,)
(28830, 31)
(28830,)


#### Building a Baseline LogisticRegression Model

In [8]:
lr = LogisticRegression(class_weight='balanced')

In [9]:
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

In [10]:
print('test set metrics: \n', metrics.classification_report(y_test, y_pred))

test set metrics: 
               precision    recall  f1-score   support

          No       0.32      0.62      0.43      7030
         Yes       0.83      0.58      0.68     21800

    accuracy                           0.59     28830
   macro avg       0.58      0.60      0.55     28830
weighted avg       0.70      0.59      0.62     28830



In [11]:
lr.get_params()

{'C': 1.0,
 'class_weight': 'balanced',
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'deprecated',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

#### Hyperparameter Tuning
Now we will be performing the tuning of hyperparameters of the LogisticRegression model. 

In [20]:
df['Status'].value_counts()

Status
Yes    108729
No      35419
Name: count, dtype: int64

In [25]:
total_count = 144148  # 108729 + 35419
weight_yes = total_count / 108729  # 1.32
weight_no = total_count / 35419  # 4.07

class_weights = {'Yes': weight_yes, 'No': weight_no}
print(class_weights)  # Виведе приблизно {'Yes': 1.32, 'No': 4.07}

{'Yes': 1.32575485840944, 'No': 4.069793048928541}


In [40]:
%%time
# Create the random grid
param_grid = {
    'penalty': ['l1', 'l2'],  # Використовуйте тільки один вид регуляризації
    'C': [0.01, 0.1, 1, 10],  # Зменште кількість значень
    'solver': ['liblinear', 'saga'],  # Використовуйте лише сумісні солвери
    'max_iter': [100, 200, 300, 500],  # Зменшіть максимальну кількість ітерацій
    'class_weight': ['balanced'],  # Ваги класів
}


lr = LogisticRegression(class_weight='balanced')

grid = GridSearchCV(estimator=lr, 
                    param_grid=param_grid, 
                    scoring='accuracy', 
                    cv=5,
                    verbose=3,
                    return_train_score=True)

grid.fit(X_train, y_train)

print("The best parameters are %s with a score of %0.2f"
      % (grid.best_params_, grid.best_score_))

Fitting 5 folds for each of 64 candidates, totalling 320 fits
[CV 1/5] END C=0.01, class_weight=balanced, max_iter=100, penalty=l1, solver=liblinear;, score=(train=0.684, test=0.684) total time=   4.0s
[CV 2/5] END C=0.01, class_weight=balanced, max_iter=100, penalty=l1, solver=liblinear;, score=(train=0.681, test=0.678) total time=   3.5s
[CV 3/5] END C=0.01, class_weight=balanced, max_iter=100, penalty=l1, solver=liblinear;, score=(train=0.684, test=0.685) total time=   5.1s
[CV 4/5] END C=0.01, class_weight=balanced, max_iter=100, penalty=l1, solver=liblinear;, score=(train=0.682, test=0.684) total time=   4.9s
[CV 5/5] END C=0.01, class_weight=balanced, max_iter=100, penalty=l1, solver=liblinear;, score=(train=0.684, test=0.683) total time=   3.8s
[CV 1/5] END C=0.01, class_weight=balanced, max_iter=100, penalty=l1, solver=saga;, score=(train=0.486, test=0.482) total time=   4.0s
[CV 2/5] END C=0.01, class_weight=balanced, max_iter=100, penalty=l1, solver=saga;, score=(train=0.483,

In [33]:
grid_results = pd.concat([pd.DataFrame(grid.cv_results_["params"]),
                          pd.DataFrame(grid.cv_results_["mean_test_score"], 
                          columns=["precision"])],
                          axis=1)

grid_results

,C,class_weight,max_iter,penalty,solver,precision
0,0.1,balanced,100,l2,liblinear,0.684949
1,0.1,balanced,100,l2,saga,0.472805
2,0.1,balanced,200,l2,liblinear,0.684949
3,0.1,balanced,200,l2,saga,0.477410
4,1.0,balanced,100,l2,liblinear,0.685504
5,1.0,balanced,100,l2,saga,0.471687
6,1.0,balanced,200,l2,liblinear,0.685504
7,1.0,balanced,200,l2,saga,0.477792


In [34]:
grid_results.columns

Index(['C', 'class_weight', 'max_iter', 'penalty', 'solver', 'precision'], dtype='object')